In [ ]:
%matplotlib inline

In [ ]:
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.mplot3d import Axes3D
import sklearn
from datetime import datetime, date
from sklearn_pandas import DataFrameMapper
import warnings
import timeit
from collections import defaultdict, OrderedDict
import tabulate
import time
import glob


timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

matplotlib.style.use('ggplot')

RANDOM_SEED = 33

## Read the CSV, parse the 24/7 and late fields into a single field

In [ ]:
restaurants = pandas.read_csv('restaurant_database.csv')

restaurants['open'] = ['24/7' if value == 'yes' else 'early' for value in restaurants['24/7?']]
restaurants['open'] = ['late' if (current == 'early' and 'open_late' == 'yes') else 'current'
                       for current, open_late 
                       in zip(restaurants['open'], restaurants['Open late (after 10pm)'])]

In [ ]:
def sanitze_to_atom(name):
    return name.lower().replace(' ', '_').replace('-', '_').replace('?', '').replace(',', '')


BOOLEAN_AFFIRMATIVE_NAME = {
    'coffee': 'served',
    'alcohol': 'served',
    'vegeterian': 'friendly',
    'pescaterian': 'friendly',
    'take_away': 'offered'
}


COLUMNS_TO_IGNORE = (
    'Name', 'Address', 'Open late (after 10pm)', '24/7?'
)


class BooleanTransformer:
    def __init__(self, col_atom):
        self.col_atom = col_atom
        
    def __call__(self, value):
        transformed = '{name}({value})'.format(name=self.col_atom, 
                                               value=BOOLEAN_AFFIRMATIVE_NAME[self.col_atom])
                
        if value in ('yes', 'y', 'True', 'true', True):
            return transformed 

        return 'not({t})'.format(t=transformed)

    
class MultivaluedTransformer:
    def __init__(self, col_atom):
        self.col_atom = col_atom
        
    def __call__(self, value):
        return '{name}({value})'.format(name=self.col_atom, 
                                        value=sanitze_to_atom(value))

    
def build_transformers(knowledge_base):
    transformers = {}
    multivalued = []
    
    for column in knowledge_base.columns:
        if column in COLUMNS_TO_IGNORE:
            continue
        
        column_atom = sanitze_to_atom(column)
        if column_atom in BOOLEAN_AFFIRMATIVE_NAME:
            transformers[column] = BooleanTransformer(column_atom)
            
        else:
            transformers[column] = MultivaluedTransformer(column_atom)
            multivalued.append(column)
            
    return transformers, multivalued


INFORMATIVE_FIELDS = ('Name', 'Address', 'Neighborhood')
            
    
def transform_kb(knowledge_base):
    transformers, multivalued = build_transformers(knowledge_base)
    restaurants = {}
    output = []
    
    for rest_id, rest_row in knowledge_base.iterrows():
        transformed_values = [transformers[col](rest_row[col]) for col in transformers]
        output.append('suggest(rest_{id}) :- {cond}.'.format(id=rest_id, 
                                                             cond=', '.join(transformed_values)))
        
        restaurants['rest_{id}'.format(id=rest_id)] = {field: rest_row[field] 
                                                       for field in INFORMATIVE_FIELDS}
    
    output.append('')
    for col in transformers:
        col_atom = sanitze_to_atom(col)
        output.append('{col}(X) :- ask({col}, X).'.format(col=col_atom))
    
    output.append('')
    output.extend(['multivalued({col}).'.format(col=sanitze_to_atom(col)) for col in multivalued])
    
    return output, restaurants
    
        

In [ ]:
output, rests = transform_kb(restaurants)

In [ ]:
print('\n'.join(output))

In [ ]:
rests

In [2]:
KB = """
% Enter your KB below this line:

:- dynamic(known/3). 

suggest(rest_0) :- neighborhood(gangnam), cuisine_type(japanese), not(coffee(served)), alcohol(served), price_range(7500_15000), not(vegeterian(friendly)), pescaterian(friendly), not(take_away(offered)), open(current).
suggest(rest_1) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), not(alcohol(served)), price_range(5000_7500), vegeterian(friendly), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_2) :- neighborhood(gangnam), cuisine_type(taiwanese), not(coffee(served)), alcohol(served), price_range(15000_25000), not(vegeterian(friendly)), pescaterian(friendly), not(take_away(offered)), open(current).
suggest(rest_3) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), not(alcohol(served)), price_range(5000_7500), not(vegeterian(friendly)), not(pescaterian(friendly)), take_away(offered), open(current).
suggest(rest_4) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), alcohol(served), price_range(7500_15000), not(vegeterian(friendly)), not(pescaterian(friendly)), not(take_away(offered)), open(current).
suggest(rest_5) :- neighborhood(gangnam), cuisine_type(italian), coffee(served), alcohol(served), price_range(15000_25000), vegeterian(friendly), pescaterian(friendly), not(take_away(offered)), open(current).
suggest(rest_6) :- neighborhood(gangnam), cuisine_type(korean), coffee(served), not(alcohol(served)), price_range(7500_15000), vegeterian(friendly), not(pescaterian(friendly)), not(take_away(offered)), open(current).
suggest(rest_7) :- neighborhood(gangnam), cuisine_type(breakfast), coffee(served), alcohol(served), price_range(7500_15000), vegeterian(friendly), pescaterian(friendly), not(take_away(offered)), open(current).
suggest(rest_8) :- neighborhood(gangnam), cuisine_type(vietnamese), not(coffee(served)), alcohol(served), price_range(7500_15000), vegeterian(friendly), not(pescaterian(friendly)), not(take_away(offered)), open(current).
suggest(rest_9) :- neighborhood(gangnam), cuisine_type(vietnamese), not(coffee(served)), alcohol(served), price_range(7500_15000), vegeterian(friendly), not(pescaterian(friendly)), not(take_away(offered)), open(current).
suggest(rest_10) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), not(alcohol(served)), price_range(7500_15000), not(vegeterian(friendly)), not(pescaterian(friendly)), not(take_away(offered)), open(current).
suggest(rest_11) :- neighborhood(gangnam), cuisine_type(japanese), not(coffee(served)), not(alcohol(served)), price_range(5000_7500), vegeterian(friendly), not(pescaterian(friendly)), take_away(offered), open(current).
suggest(rest_12) :- neighborhood(gangnam), cuisine_type(vietnamese), not(coffee(served)), not(alcohol(served)), price_range(5000_7500), not(vegeterian(friendly)), not(pescaterian(friendly)), take_away(offered), open(current).
suggest(rest_13) :- neighborhood(gangnam), cuisine_type(japanese), not(coffee(served)), alcohol(served), price_range(25000+), vegeterian(friendly), not(pescaterian(friendly)), not(take_away(offered)), open(current).
suggest(rest_14) :- neighborhood(gangnam), cuisine_type(american), coffee(served), alcohol(served), price_range(7500_15000), not(vegeterian(friendly)), not(pescaterian(friendly)), take_away(offered), open(current).
suggest(rest_15) :- neighborhood(gangnam), cuisine_type(japanese), not(coffee(served)), alcohol(served), price_range(7500_15000), vegeterian(friendly), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_16) :- neighborhood(gangnam), cuisine_type(japanese), not(coffee(served)), not(alcohol(served)), price_range(7500_15000), vegeterian(friendly), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_17) :- neighborhood(gangnam), cuisine_type(american), coffee(served), not(alcohol(served)), price_range(7500_15000), vegeterian(friendly), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_18) :- neighborhood(gangnam), cuisine_type(american), coffee(served), not(alcohol(served)), price_range(7500_15000), not(vegeterian(friendly)), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_19) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), alcohol(served), price_range(7500_15000), not(vegeterian(friendly)), not(pescaterian(friendly)), take_away(offered), open(current).
suggest(rest_20) :- neighborhood(gangnam), cuisine_type(chinese), not(coffee(served)), alcohol(served), price_range(7500_15000), vegeterian(friendly), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_21) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), alcohol(served), price_range(7500_15000), not(vegeterian(friendly)), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_22) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), alcohol(served), price_range(7500_15000), not(vegeterian(friendly)), not(pescaterian(friendly)), not(take_away(offered)), open(current).
suggest(rest_23) :- neighborhood(itaewon), cuisine_type(turkish), not(coffee(served)), not(alcohol(served)), price_range(5000_7500), not(vegeterian(friendly)), not(pescaterian(friendly)), take_away(offered), open(current).
suggest(rest_24) :- neighborhood(gangnam), cuisine_type(american), coffee(served), not(alcohol(served)), price_range(15000_25000), vegeterian(friendly), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_25) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), alcohol(served), price_range(7500_15000), not(vegeterian(friendly)), not(pescaterian(friendly)), not(take_away(offered)), open(current).
suggest(rest_26) :- neighborhood(gangnam), cuisine_type(korean), not(coffee(served)), alcohol(served), price_range(5000_7500), vegeterian(friendly), pescaterian(friendly), take_away(offered), open(current).
suggest(rest_27) :- neighborhood(gangnam), cuisine_type(japanese), not(coffee(served)), not(alcohol(served)), price_range(7500_15000), vegeterian(friendly), pescaterian(friendly), take_away(offered), open(current).

neighborhood(X) :- ask(neighborhood, X).
cuisine_type(X) :- ask(cuisine_type, X).
coffee(X) :- ask(coffee, X).
alcohol(X) :- ask(alcohol, X).
price_range(X) :- ask(price_range, X).
vegeterian(X) :- ask(vegeterian, X).
pescaterian(X) :- ask(pescaterian, X).
take_away(X) :- ask(take_away, X).
open(X) :- ask(open, X).

multivalued(neighborhood).
multivalued(cuisine_type).
multivalued(price_range).
multivalued(open).

% The code below implements the prompting to ask the user:


% Asking clauses

multivalued(none).

ask(A, V):-
known(y, A, V), % succeed if true
!.	% stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

ask(A, V):-
not(multivalued(A)),
write_py(A:not_multivalued),
known(y, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
Y == y. % succeed or fail
"""

with open("KB_A.pl", "w") as text_file:
    text_file.write(KB)

In [3]:
# The code here will ask the user for input based on the askables
# It will check if the answer is known first

from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    Y.unify(input(str(A) + " is " + str(V) + "? "))
    return True


write_py.arity = 1
read_py.arity = 3

registerForeign(read_py)
registerForeign(write_py)

prolog.consult("KB_A.pl") # open the KB
call(retractall(known))
for soln in prolog.query("suggest(X).", maxresult=1):
    print("Your restaurant reccomendation is " + soln['X'])

neighborhood is gangnam? yes


KeyboardInterrupt: 

PrologError: Caused by: 'suggest(X).'. Returned: 'error(domain_error(foreign_return_value, 140259608048808), context(/(read_py, 3), _2018))'.